In [1]:
import numpy as np
import psana as ps
import matplotlib.pyplot as plt
import sys
import time
import pickle
from Functions import *
from raw_data_class import RawData as RDC
%matplotlib notebook

In [2]:
save_dir = '/cds/home/b/bpoult/LCLS_LV_27_Data_Analysis/sample_data/'
runs = list(range(207,214))
epix_roi = [80,140,170,330]
xrt_roi = [725,1500]

# Processing or Loading
process_raw = False
on_off = False
plot_one = 210
x_axis = 'energy' # 'pixels' or 'energy'

# Energy Calibration, plot raw calibration runs with pixel axis and populate these arrays
save_cal = True
labels = ['notch_energies','xrt_pixels','epix_pixels']
notch_energies = [7.06,7.065,7.07,7.075,7.08,7.085,7.09] # keV
xrt_pixels = [1474,1385,1297,1216,1131,1048,966] # enter 0 if you can't see the notch
epix_pixels = [0,223,236,248,262,274,0] # enter 0 if you can't see the notch
calib_data = [labels, notch_energies,xrt_pixels,epix_pixels] 

In [13]:
# Raw Data
raw_datas = []
for run in runs:
    scan_name = 'run_' + f'{run:03}'
    ds_string = 'exp=cxix46119:run='+str(run)+':smd'
    print(ds_string)
    if process_raw:
        raw_data = load_data.load_data(save_dir,scan_name,ds_string,epix_roi,xrt_roi)
    else:
        with open(save_dir + scan_name + "/" + "rawdata.pkl", "rb") as f:
            raw_data = pickle.load(f)
            
    raw_datas = raw_datas + [raw_data]

plot_raw.plot_raw(raw_datas, plot_one, x_axis, on_off)

exp=cxix46119:run=207:smd
exp=cxix46119:run=208:smd
exp=cxix46119:run=209:smd
exp=cxix46119:run=210:smd
exp=cxix46119:run=211:smd
exp=cxix46119:run=212:smd
exp=cxix46119:run=213:smd
Raw Data plotting is turned off.


In [16]:
# Energy Calibration
raw_datas = [energy_calib.energy_calib(raw_datas[i],calib_data,save_cal) for i in range(0,len(raw_datas))]

['notch_energies', 'xrt_pixels', 'epix_pixels', 'xrt_energy', 'epix_energy', 'time_calibrated']
